# Documentation
https://www.notion.so/huthvincent/Code-documentation-of-Rui-9f0b3ea397864c8b9b2401926a3b90f1

# Package

In [1]:
import argparse
import logging
import pdb
import sys
import time
import math
from tqdm import tqdm
import os
import copy
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from utils_notion import *
from datetime import datetime

from wideresnet import WideResNet
from preactresnet import PreActResNet18
from utils import *
from utils_awp import AdvWeightPerturb

mu = torch.tensor(cifar10_mean).view(3, 1, 1).cuda()
std = torch.tensor(cifar10_std).view(3, 1, 1).cuda()


# Variable

In [2]:
training_type = "Enhancement" # {pretrain, awp, Enhancement}
model_name = 'PreActResNet18'

## Pretrain

In [3]:
if training_type == "pretrain":
    l2 = 0
    l1 = 0
    batch_size = 128 
    batch_size_test = 128 
    data_dir ='./cifar-data' 
    epochs =100 
    lr_schedule = 'piecewise'
    # choices=['superconverge', 'piecewise', 'linear', 'piecewisesmoothed', 'piecewisezoom', 'onedrop', 'multipledecay', 'cosine', 'cyclic'])
    lr_max =0.1
    lr_one_drop =0.01
    lr_drop_epoch =100
    AE_attack ='none' #type=str, choices=['pgd', 'fgsm', 'free', 'none'])
    pretrain_iter = 0
    epsilon =128
    attack_iters =10
    attack_iters_test =20
    restarts =1
    pgd_alpha =2
    fgsm_alpha =12
    norm ='l_2'#choices=['l_inf', 'l_2']
    fgsm_init ='random' #choices=['zero', 'random', 'previous']
    fname ='models/' + training_type + '_cifar_model'
    seed =0
    half =True
    width_factor =10
    resume = 0
    cutout =False
    cutout_len = 10
    mixup =True
    mixup_alpha = 0
    eval_ =False
    val =True
    chkpt_iters = 10
    awp_gamma =0.01
    awp_warmup = 100
    enhancement = False
    

## AWP

In [4]:
if training_type == "AWP":
    l2 = 0
    l1 = 0
    batch_size = 128 
    batch_size_test = 128 
    data_dir ='./cifar-data' 
    epochs =200 
    lr_schedule = 'piecewise'
    # choices=['superconverge', 'piecewise', 'linear', 'piecewisesmoothed', 'piecewisezoom', 'onedrop', 'multipledecay', 'cosine', 'cyclic'])
    lr_max =0.1
    lr_one_drop =0.01
    lr_drop_epoch =100
    AE_attack ='pgd' #type=str, choices=['pgd', 'fgsm', 'free', 'none'])
    pretrain_iter = 50
    epsilon =128
    attack_iters =10
    attack_iters_test =20
    restarts =1
    pgd_alpha =2
    fgsm_alpha =12
    norm ='l_2'#choices=['l_inf', 'l_2']
    fgsm_init ='random' #choices=['zero', 'random', 'previous']
    fname ='models/' + training_type + '_cifar_model'
    seed =0
    half =True
    width_factor =10
    resume = 0
    cutout =False
    cutout_len = 10
    mixup =False
    mixup_alpha = 0.1
    eval_ =False
    val =True
    chkpt_iters = 10
    awp_gamma =0.01
    awp_warmup = 60
    enhancement = False

## Enhancement

In [5]:
if training_type == "Enhancement":
    l2 = 0
    l1 = 0
    batch_size = 128 
    batch_size_test = 128 
    data_dir ='./cifar-data' 
    epochs = 600 
    lr_schedule = 'superconverge'
    # choices=['superconverge', 'piecewise', 'linear', 'piecewisesmoothed', 'piecewisezoom', 'onedrop', 'multipledecay', 'cosine', 'cyclic'])
    lr_max =0.1
    lr_one_drop =0.01
    lr_drop_epoch =100
    AE_attack ='pgd' #type=str, choices=['pgd', 'fgsm', 'free', 'none'])
    pretrain_iter = 600
    epsilon =128
    attack_iters =10
    attack_iters_test =20
    restarts =1
    pgd_alpha =2
    fgsm_alpha =12
    norm ='l_2'#choices=['l_inf', 'l_2']
    fgsm_init ='random' #choices=['zero', 'random', 'previous']
    
    seed =0
    half =True
    width_factor =10
    resume = 0
    cutout = False
    cutout_len = 10
    mixup =False
    mixup_alpha = 0
    eval_ =False
    val =True
    chkpt_iters = 10
    awp_gamma =0.01
    awp_warmup = 1000
    enhancement = True
    enhancement_warmup = 10
    pull = "away" #[in, none, away]
    scale = 10 
    num_layer_modify = 17
    training_type = pull + "_enhancement"
    fname ="/home/rui/Desktop/disk2/data/trojan_casual/models/notion_management_models/"

# Methods

In [6]:

def normalize(X):
    return (X - mu)/std


upper_limit, lower_limit = 1,0


def clamp(X, lower_limit, upper_limit):
    return torch.max(torch.min(X, upper_limit), lower_limit)


class Batches():
    def __init__(self, dataset, batch_size, shuffle, set_random_choices=False, num_workers=0, drop_last=False):
        self.dataset = dataset
        self.batch_size = batch_size
        self.set_random_choices = set_random_choices
        self.dataloader = torch.utils.data.DataLoader(
            dataset, batch_size=batch_size, num_workers=num_workers, pin_memory=True, shuffle=shuffle, drop_last=drop_last
        )

    def __iter__(self):
        if self.set_random_choices:
            self.dataset.set_random_choices()
        return ({'input': x.to(device).float(), 'target': y.to(device).long()} for (x,y) in self.dataloader)

    def __len__(self):
        return len(self.dataloader)


def mixup_data(x, y, alpha=1.0):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    index = torch.randperm(batch_size).cuda()

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam


def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)


def attack_pgd(model, X, y, epsilon, alpha, attack_iters, restarts,
               norm, early_stop=False,
               mixup=False, y_a=None, y_b=None, lam=None):
    max_loss = torch.zeros(y.shape[0]).cuda()
    max_delta = torch.zeros_like(X).cuda()
    for _ in range(restarts):
        delta = torch.zeros_like(X).cuda()
        if norm == "l_inf":
            delta.uniform_(-epsilon, epsilon)
        elif norm == "l_2":
            delta.normal_()
            d_flat = delta.view(delta.size(0),-1)
            n = d_flat.norm(p=2,dim=1).view(delta.size(0),1,1,1)
            r = torch.zeros_like(n).uniform_(0, 1)
            delta *= r/n*epsilon
        else:
            raise ValueError
        delta = clamp(delta, lower_limit-X, upper_limit-X)
        delta.requires_grad = True
        for _ in range(attack_iters):
            output = model(normalize(X + delta))
            if early_stop:
                # only construct adversarial samples for the correctly-classified
                index = torch.where(output.max(1)[1] == y)[0]
            else:
                index = slice(None,None,None)
            if not isinstance(index, slice) and len(index) == 0:
                break
            if mixup:
                criterion = nn.CrossEntropyLoss()
                loss = mixup_criterion(criterion, model(normalize(X+delta)), y_a, y_b, lam)
            else:
                loss = F.cross_entropy(output, y)
            loss.backward()
            grad = delta.grad.detach()
            d = delta[index, :, :, :]
            g = grad[index, :, :, :]
            x = X[index, :, :, :]
            if norm == "l_inf":
                d = torch.clamp(d + alpha * torch.sign(g), min=-epsilon, max=epsilon)
            elif norm == "l_2":
                g_norm = torch.norm(g.view(g.shape[0],-1),dim=1).view(-1,1,1,1)
                scaled_g = g/(g_norm + 1e-10)
                d = (d + scaled_g*alpha).view(d.size(0),-1).renorm(p=2,dim=0,maxnorm=epsilon).view_as(d)
            d = clamp(d, lower_limit - x, upper_limit - x)
            delta.data[index, :, :, :] = d
            delta.grad.zero_()
        if mixup:
            criterion = nn.CrossEntropyLoss(reduction='none')
            all_loss = mixup_criterion(criterion, model(normalize(X+delta)), y_a, y_b, lam)
        else:
            all_loss = F.cross_entropy(model(normalize(X+delta)), y, reduction='none')
        max_delta[all_loss >= max_loss] = delta.detach()[all_loss >= max_loss]
        max_loss = torch.max(max_loss, all_loss)
    return max_delta






In [7]:
def config_lr_scheduler(lr_schedule):
    if lr_schedule == 'superconverge':
        lr_schedule = lambda t: np.interp([t], [0, epochs * 2 // 5, epochs], [0, lr_max, 0])[0]
    elif lr_schedule == 'piecewise':
        def lr_schedule(t):
            if t / epochs < 0.5:
                return lr_max
            elif t / epochs < 0.75:
                return lr_max / 10.
            else:
                return lr_max / 100.
    elif lr_schedule == 'linear':
        lr_schedule = lambda t: np.interp([t], [0, epochs//3, epochs*2//3,epochs],[lr_max, lr_max, lr_max / 10, lr_max / 100])[0]
    elif lr_schedule == 'onedrop':
        def lr_schedule(t):
            if t < lr_drop_epoch:
                return lr_max
            else:
                return lr_one_drop
    elif lr_schedule == 'multipledecay':
        def lr_schedule(t):
            return lr_max - (t//(epochs//10))*(lr_max/10)
    elif lr_schedule == 'cosine':
        def lr_schedule(t):
            return lr_max * 0.5 * (1 + np.cos(t / epochs * np.pi))
    elif lr_schedule == 'cyclic':
        lr_schedule = lambda t: np.interp([t], [0, 0.4 * epochs, epochs], [0, lr_max, 0])[0]
    return lr_schedule


In [8]:
config_lr_scheduler(lr_schedule)

<function __main__.config_lr_scheduler.<locals>.<lambda>(t)>

In [9]:
if not os.path.exists(fname):
    os.makedirs(fname)
# logger = logging.getLogger(__name__)
# logging.basicConfig(
#     format='[%(asctime)s] - %(message)s',
#     datefmt='%Y/%m/%d %H:%M:%S',
#     level=logging.DEBUG,
#     handlers=[
#         logging.FileHandler(os.path.join(fname, 'eval.log' if eval_ else 'output.log')),
#         logging.StreamHandler()
#     ])

# logger.info(args)

np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

# logger

In [10]:
logger = logging.getLogger()
logging.basicConfig(
    format='[%(asctime)s] - %(message)s',
    datefmt='%Y/%m/%d %H:%M:%S',
    level=logging.DEBUG,
    handlers=[
        logging.FileHandler(os.path.join(fname, 'eval.log' if eval_ else 'output.log')),
        logging.StreamHandler()
    ])

# dataset

In [11]:
transforms = [Crop(32, 32), FlipLR()]
if cutout:
    transforms.append(Cutout(cutout_len, cutout_len))

dataset = cifar10(data_dir, True)
train_set = list(zip(transpose(pad(dataset['train']['data'], 4)/255.),
    dataset['train']['labels']))
train_set_x = Transform(train_set, transforms)
train_batches = Batches(train_set_x, batch_size, shuffle=True, set_random_choices=True, num_workers=2)

val_set = list(zip(transpose(dataset['val']['data']/255.), dataset['val']['labels']))
val_batches = Batches(val_set, batch_size, shuffle=False, num_workers=2)

test_set = list(zip(transpose(dataset['test']['data']/255.), dataset['test']['labels']))
test_batches = Batches(test_set, batch_size_test, shuffle=False, num_workers=2)

epsilon = (epsilon / 255.)
pgd_alpha = (pgd_alpha / 255.)

Files already downloaded and verified
Files already downloaded and verified


# Model initialization

In [12]:
class PullNoisePerturb(object):
    def __init__(self, model, proxy):
        super(PullNoisePerturb, self).__init__()
        self.model = model
        self.proxy = proxy

    def add_pertube(self, pull, scale, num_layer_modify):
        std = 0.1
        with torch.no_grad():
                conv_layer_cter = 0
                std = std * 0.9
                for para in model.parameters():
                    if len(para.shape) == 4:
                        if num_layer_modify > conv_layer_cter:
                            para_mean = torch.mean(para).cpu()
#                             noise = torch.randn(para.size()) * scale * para_mean
                            noise = torch.randn(para.size()) * std
                            for s1 in range(len(para)):
                                for s2 in range(len(para[s1])):
                                    for s3 in range(len(para[s1][s2])):
                                        for s4 in range(len(para[s1][s2][s3])):
                                            if para[s1][s2][s3][s4] > para_mean:
                                                if pull == "away":
                                                    noise[s1][s2][s3][s4] = abs(noise[s1][s2][s3][s4])
                                                elif pull == "in":
                                                    noise[s1][s2][s3][s4] = -abs(noise[s1][s2][s3][s4])

                                            else:
                                                if pull == "away":
                                                    noise[s1][s2][s3][s4] = -abs(noise[s1][s2][s3][s4])
                                                elif pull == "in":
                                                    noise[s1][s2][s3][s4] = abs(noise[s1][s2][s3][s4])

                            para.add_((noise).to(device))
                        conv_layer_cter = conv_layer_cter + 1


In [13]:
if model_name == 'PreActResNet18':
    model = PreActResNet18()
    proxy = PreActResNet18()
elif model_name == 'WideResNet':
    model = WideResNet(28, 10, widen_factor=width_factor, dropRate=0.0)
    proxy = WideResNet(28, 10, widen_factor=width_factor, dropRate=0.0)
else:
    raise ValueError("Unknown model")

model = nn.DataParallel(model).cuda()
proxy = nn.DataParallel(proxy).cuda()
# model.to(device)
# proxy.to(device)

if l2:
    decay, no_decay = [], []
    for name,param in model.named_parameters():
        if 'bn' not in name and 'bias' not in name:
            decay.append(param)
        else:
            no_decay.append(param)
    params = [{'params':decay, 'weight_decay':l2},
              {'params':no_decay, 'weight_decay': 0 }]
else:
    params = model.parameters()

opt = torch.optim.SGD(params, lr=lr_max, momentum=0.9, weight_decay=5e-4)
proxy_opt = torch.optim.SGD(proxy.parameters(), lr=0.01)
awp_adversary = AdvWeightPerturb(model=model, proxy=proxy, proxy_optim=proxy_opt, gamma=awp_gamma)
pull_noise_adversary = PullNoisePerturb(model=model, proxy=proxy)
criterion = nn.CrossEntropyLoss()

# Config attack

In [14]:

if  AE_attack== 'free':
    delta = torch.zeros(batch_size, 3, 32, 32).cuda()
    delta.requires_grad = True
elif AE_attack == 'fgsm' and fgsm_init == 'previous':
    delta = torch.zeros(batch_size, 3, 32, 32).cuda()
    delta.requires_grad = True

lr_schedule = config_lr_scheduler(lr_schedule)

best_val_robust_loss = math.inf
if resume:
    start_epoch = resume
    state_resumed = torch.load(os.path.join(fname, f'state_{start_epoch-1}.pth'))
    model.load_state_dict(state_resumed['model_state'])
#     opt.load_state_dict(state_resumed['opt_state'])
    logger.info(f'Resuming at epoch {start_epoch}')

    best_val_robust_loss = state_resumed['val_robust_loss']
else:
    start_epoch = 0

epochs = epochs + start_epoch

# Training or evaluation

In [15]:
if eval_:
    if not resume:
        logger.info("No model loaded to evaluate, specify with --resume FNAME")
    logger.info("[Evaluation mode]")

if not eval_:
    ID = "".join(list(filter(str.isdigit, str(datetime.now()))))
    logger.info(f'{"="*20} Train {"="*20}')
    logger.info('Epoch \t Time Elapse \t LR \t \t Loss \t \t  Acc \t \t Robust Loss \t \t Robust Acc')
    for epoch in range(start_epoch, epochs):
        if epoch > pretrain_iter:
            attack = AE_attack
        else:
            attack = 'none'
        start_time = time.time()
        train_loss = 0
        train_acc = 0
        train_robust_loss = 0
        train_robust_acc = 0
        train_n = 0
        model.train()
        for i, batch in enumerate(tqdm(train_batches)):
            X, y = batch['input'], batch['target']
            if mixup:
                X, y_a, y_b, lam = mixup_data(X, y, mixup_alpha)
                X, y_a, y_b = map(Variable, (X, y_a, y_b))
#             lr = lr_schedule(epoch + (i + 1) / len(train_batches))
            lr = 0.005
            opt.param_groups[0].update(lr=lr)

            if attack == 'pgd':
                # Random initialization
                if mixup:
                    delta = attack_pgd(model, X, y, epsilon, pgd_alpha, attack_iters, restarts, norm, mixup=True, y_a=y_a, y_b=y_b, lam=lam)
                else:
                    delta = attack_pgd(model, X, y, epsilon, pgd_alpha, attack_iters, restarts, norm)
                delta = delta.detach()
            elif attack == 'fgsm':
                delta = attack_pgd(model, X, y, epsilon, fgsm_alpha*epsilon, 1, 1, norm)
            # Standard training
            elif attack == 'none':
                delta = torch.zeros_like(X)
            X_adv = normalize(torch.clamp(X + delta[:X.size(0)], min=lower_limit, max=upper_limit))

            # calculate adversarial weight perturbation and perturb it
            if epoch >= awp_warmup:
                # not compatible to mixup currently.
                assert (not mixup)
                awp = awp_adversary.calc_awp(inputs_adv=X_adv,
                                            targets=y)
                awp_adversary.perturb(awp)
            
            robust_output = model(X_adv)
            if mixup:
                robust_loss = mixup_criterion(criterion, robust_output, y_a, y_b, lam)
            else:
                robust_loss = criterion(robust_output, y)

            # L1 regularization
            if l1:
                for name,param in model.named_parameters():
                    if 'bn' not in name and 'bias' not in name:
                        robust_loss += l1*param.abs().sum()

            opt.zero_grad()
            robust_loss.backward()
            opt.step()

            if epoch >= awp_warmup:
                awp_adversary.restore(awp)
            
            output = model(normalize(X))
            if mixup:
                loss = mixup_criterion(criterion, output, y_a, y_b, lam)
            else:
                loss = criterion(output, y)

            train_robust_loss += robust_loss.item() * y.size(0)
            train_robust_acc += (robust_output.max(1)[1] == y).sum().item()
            train_loss += loss.item() * y.size(0)
            train_acc += (output.max(1)[1] == y).sum().item()
            train_n += y.size(0)

        train_time = time.time()
        logger.info('%d \t %.1f \t \t %.4f \t %.4f \t  %.4f \t %.4f \t \t %.4f',
            epoch, train_time - start_time, lr,
            train_loss/train_n, train_acc/train_n, train_robust_loss/train_n, train_robust_acc/train_n)

        # save checkpoint upon validation
        if (epoch+1) % chkpt_iters == 0 or epoch+1 == epochs:
            model.eval()
            val_loss = 0
            val_acc = 0
            val_robust_loss = 0
            val_robust_acc = 0
            val_n = 0
            for i, batch in enumerate(tqdm(val_batches)):
                X, y = batch['input'], batch['target']

                # Random initialization
#                 if attack == 'none':
#                     delta = torch.zeros_like(X)
#                 else:
#                     delta = attack_pgd(model, X, y, epsilon, pgd_alpha, attack_iters_test, restarts, norm, early_stop=eval_)
                delta = attack_pgd(model, X, y, epsilon, pgd_alpha, attack_iters_test, restarts, norm, early_stop=eval_)
                delta = delta.detach()

                robust_output = model(normalize(torch.clamp(X + delta[:X.size(0)], min=lower_limit, max=upper_limit)))
                robust_loss = criterion(robust_output, y)

                output = model(normalize(X))
                loss = criterion(output, y)

                val_robust_loss += robust_loss.item() * y.size(0)
                val_robust_acc += (robust_output.max(1)[1] == y).sum().item()
                val_loss += loss.item() * y.size(0)
                val_acc += (output.max(1)[1] == y).sum().item()
                val_n += y.size(0)

            logger.info(f'{"="*20} Validation {"="*20}')
            logger.info('Loss \t Acc \t Robust Loss \t Robust Acc')
            logger.info('%.4f \t %.4f \t %.4f \t %.4f', val_loss/val_n, val_acc/val_n, val_robust_loss/val_n, val_robust_acc/val_n)

            if val_robust_loss/val_n < best_val_robust_loss:
                torch.save({
                        'model_state':model.state_dict(),
                        'opt_state':opt.state_dict(),
                        'val_robust_acc':val_robust_acc/val_n,
                        'val_robust_loss':val_robust_loss/val_n
                    }, os.path.join(fname, f'state_{epoch}.pth'))
                best_val_robust_loss = val_robust_loss/val_n
                
            if enhancement:
                pull_noise_adversary.add_pertube(pull, scale, num_layer_modify)
            print("*********************************************************")
            print("After pertubation")
            print("*********************************************************")
            model.eval()

            val_loss = 0
            val_acc = 0
            val_robust_loss = 0
            val_robust_acc = 0
            val_n = 0
            for i, batch in enumerate(tqdm(val_batches)):
                X, y = batch['input'], batch['target']

                # Random initialization
#                 if attack == 'none':
#                     delta = torch.zeros_like(X)
#                 else:
#                     delta = attack_pgd(model, X, y, epsilon, pgd_alpha, attack_iters_test, restarts, norm, early_stop=eval_)
                delta = attack_pgd(model, X, y, epsilon, pgd_alpha, attack_iters_test, restarts, norm, early_stop=eval_)
                delta = delta.detach()

                robust_output = model(normalize(torch.clamp(X + delta[:X.size(0)], min=lower_limit, max=upper_limit)))
                robust_loss = criterion(robust_output, y)

                output = model(normalize(X))
                loss = criterion(output, y)

                val_robust_loss += robust_loss.item() * y.size(0)
                val_robust_acc += (robust_output.max(1)[1] == y).sum().item()
                val_loss += loss.item() * y.size(0)
                val_acc += (output.max(1)[1] == y).sum().item()
                val_n += y.size(0)

            logger.info(f'{"="*20} Validation {"="*20}')
            logger.info('Loss \t Acc \t Robust Loss \t Robust Acc')
            logger.info('%.4f \t %.4f \t %.4f \t %.4f', val_loss/val_n, val_acc/val_n, val_robust_loss/val_n, val_robust_acc/val_n)

            if val_robust_loss/val_n < best_val_robust_loss:
                torch.save({
                        'model_state':model.state_dict(),
                        'opt_state':opt.state_dict(),
                        'val_robust_acc':val_robust_acc/val_n,
                        'val_robust_loss':val_robust_loss/val_n
                    }, os.path.join(fname + ID + "_", f'state_{epoch}.pth'))
                best_val_robust_loss = val_robust_loss/val_n
    
    model_info_dict = {"ID" : ID,
                   "Dataset" : "CIFAR-10",
                   "Architecture" : "VGG-16",
                   "Backdoor" : "BadNet",
                   "AT" : "Pull_away"}
    createPage(databaseId, headers, model_info_dict)
    logger.info('Finish Training')


    

[2022/12/19 13:40:50] - ==================== Train ====================
[2022/12/19 13:40:50] - Epoch 	 Time Elapse 	 LR 	 	 Loss 	 	  Acc 	 	 Robust Loss 	 	 Robust Acc
100%|█████████████████████████████████████████| 313/313 [00:17<00:00, 17.75it/s]
[2022/12/19 13:41:08] - 0 	 17.6 	 	 0.0050 	 1.4073 	  0.4914 	 1.4750 	 	 0.4592
100%|█████████████████████████████████████████| 313/313 [00:15<00:00, 19.82it/s]
[2022/12/19 13:41:24] - 1 	 15.8 	 	 0.0050 	 0.9238 	  0.6734 	 1.0053 	 	 0.6414
100%|█████████████████████████████████████████| 313/313 [00:15<00:00, 19.68it/s]
[2022/12/19 13:41:40] - 2 	 15.9 	 	 0.0050 	 0.7144 	  0.7500 	 0.7951 	 	 0.7177
100%|█████████████████████████████████████████| 313/313 [00:15<00:00, 19.60it/s]
[2022/12/19 13:41:56] - 3 	 16.0 	 	 0.0050 	 0.5920 	  0.7973 	 0.6705 	 	 0.7651
100%|█████████████████████████████████████████| 313/313 [00:15<00:00, 19.64it/s]
[2022/12/19 13:42:12] - 4 	 15.9 	 	 0.0050 	 0.5044 	  0.8274 	 0.5795 	 	 0.7976
100%|█████

*********************************************************
After pertubation
*********************************************************


100%|███████████████████████████████████████████| 79/79 [01:03<00:00,  1.25it/s]
[2022/12/19 13:48:42] - ==================== Validation ====================
[2022/12/19 13:48:42] - Loss 	 Acc 	 Robust Loss 	 Robust Acc
[2022/12/19 13:48:42] - 1539776571.3920 	 0.0998 	 1664413726.9248 	 0.0998
100%|█████████████████████████████████████████| 313/313 [00:16<00:00, 19.54it/s]
[2022/12/19 13:48:58] - 10 	 16.0 	 	 0.0050 	 0.7606 	  0.7565 	 0.7762 	 	 0.7499
100%|█████████████████████████████████████████| 313/313 [00:15<00:00, 19.59it/s]
[2022/12/19 13:49:14] - 11 	 16.0 	 	 0.0050 	 0.4978 	  0.8305 	 0.5115 	 	 0.8245
100%|█████████████████████████████████████████| 313/313 [00:16<00:00, 19.51it/s]
[2022/12/19 13:49:30] - 12 	 16.0 	 	 0.0050 	 0.4415 	  0.8495 	 0.4563 	 	 0.8443
100%|█████████████████████████████████████████| 313/313 [00:16<00:00, 19.52it/s]
[2022/12/19 13:49:46] - 13 	 16.0 	 	 0.0050 	 0.4069 	  0.8596 	 0.4218 	 	 0.8542
100%|███████████████████████████████████████

*********************************************************
After pertubation
*********************************************************


100%|███████████████████████████████████████████| 79/79 [01:01<00:00,  1.28it/s]
[2022/12/19 13:56:34] - ==================== Validation ====================
[2022/12/19 13:56:34] - Loss 	 Acc 	 Robust Loss 	 Robust Acc
[2022/12/19 13:56:34] - 2942.3490 	 0.0998 	 3339.7405 	 0.0998
100%|█████████████████████████████████████████| 313/313 [00:15<00:00, 19.95it/s]
[2022/12/19 13:56:49] - 20 	 15.7 	 	 0.0050 	 0.4435 	  0.8495 	 0.4510 	 	 0.8463
100%|█████████████████████████████████████████| 313/313 [00:15<00:00, 19.89it/s]
[2022/12/19 13:57:05] - 21 	 15.7 	 	 0.0050 	 0.3889 	  0.8681 	 0.3957 	 	 0.8656
100%|█████████████████████████████████████████| 313/313 [00:15<00:00, 19.67it/s]
[2022/12/19 13:57:21] - 22 	 15.9 	 	 0.0050 	 0.3675 	  0.8730 	 0.3746 	 	 0.8700
100%|█████████████████████████████████████████| 313/313 [00:16<00:00, 18.72it/s]
[2022/12/19 13:57:38] - 23 	 16.7 	 	 0.0050 	 0.3558 	  0.8782 	 0.3630 	 	 0.8756
100%|█████████████████████████████████████████| 313/313 

...

*********************************************************
After pertubation
*********************************************************


100%|███████████████████████████████████████████| 79/79 [00:58<00:00,  1.34it/s]
[2022/12/19 17:38:00] - ==================== Validation ====================
[2022/12/19 17:38:00] - Loss 	 Acc 	 Robust Loss 	 Robust Acc
[2022/12/19 17:38:00] - 1.9430 	 0.5265 	 7.7967 	 0.1084
100%|█████████████████████████████████████████| 313/313 [00:14<00:00, 21.13it/s]
[2022/12/19 17:38:15] - 320 	 14.8 	 	 0.0050 	 0.2421 	  0.9186 	 0.2774 	 	 0.9020
100%|█████████████████████████████████████████| 313/313 [00:14<00:00, 21.12it/s]
[2022/12/19 17:38:30] - 321 	 14.8 	 	 0.0050 	 0.2336 	  0.9216 	 0.2685 	 	 0.9065
100%|█████████████████████████████████████████| 313/313 [00:14<00:00, 21.06it/s]
[2022/12/19 17:38:45] - 322 	 14.9 	 	 0.0050 	 0.2337 	  0.9205 	 0.2680 	 	 0.9062
100%|█████████████████████████████████████████| 313/313 [00:14<00:00, 21.08it/s]
[2022/12/19 17:39:00] - 323 	 14.9 	 	 0.0050 	 0.2315 	  0.9220 	 0.2665 	 	 0.9072
100%|█████████████████████████████████████████| 313/313 [0

*********************************************************
After pertubation
*********************************************************


100%|███████████████████████████████████████████| 79/79 [00:58<00:00,  1.34it/s]
[2022/12/19 17:45:26] - ==================== Validation ====================
[2022/12/19 17:45:26] - Loss 	 Acc 	 Robust Loss 	 Robust Acc
[2022/12/19 17:45:26] - 1.5432 	 0.6022 	 7.3825 	 0.1126
100%|█████████████████████████████████████████| 313/313 [00:14<00:00, 21.10it/s]
[2022/12/19 17:45:41] - 330 	 14.8 	 	 0.0050 	 0.2300 	  0.9227 	 0.2658 	 	 0.9067
100%|█████████████████████████████████████████| 313/313 [00:14<00:00, 21.09it/s]
[2022/12/19 17:45:56] - 331 	 14.8 	 	 0.0050 	 0.2229 	  0.9247 	 0.2591 	 	 0.9080
100%|█████████████████████████████████████████| 313/313 [00:14<00:00, 21.07it/s]
[2022/12/19 17:46:11] - 332 	 14.9 	 	 0.0050 	 0.2235 	  0.9241 	 0.2594 	 	 0.9085
100%|█████████████████████████████████████████| 313/313 [00:14<00:00, 21.13it/s]
[2022/12/19 17:46:26] - 333 	 14.8 	 	 0.0050 	 0.2254 	  0.9254 	 0.2614 	 	 0.9082
100%|█████████████████████████████████████████| 313/313 [0

KeyboardInterrupt: 

# Eval

In [ ]:
    

model.eval()
test_loss = 0
test_acc = 0
test_robust_loss = 0
test_robust_acc = 0
test_n = 0
for i, batch in enumerate(tqdm(test_batches)):
    X, y = batch['input'], batch['target']

    # Random initialization
    if AE_attack == 'none':
        delta = torch.zeros_like(X)
    else:
        delta = attack_pgd(model, X, y, epsilon, pgd_alpha, attack_iters_test, restarts, norm, early_stop=eval_)
    delta = delta.detach()

    robust_output = model(normalize(torch.clamp(X + delta[:X.size(0)], min=lower_limit, max=upper_limit)))
    robust_loss = criterion(robust_output, y)

    output = model(normalize(X))
    loss = criterion(output, y)

    test_robust_loss += robust_loss.item() * y.size(0)
    test_robust_acc += (robust_output.max(1)[1] == y).sum().item()
    test_loss += loss.item() * y.size(0)
    test_acc += (output.max(1)[1] == y).sum().item()
    test_n += y.size(0)

logger.info(f'{"="*20} Test {"="*20}')
logger.info('Loss \t Acc \t Robust Loss \t Robust Acc')
logger.info('%.4f \t %.4f \t %.4f \t %.4f', test_loss/test_n, test_acc/test_n, test_robust_loss/test_n, test_robust_acc/test_n)

# Import VGG

In [ ]:
# net = models_lib.vgg16(pretrained=False, progress=False, num_classes=10)
# net._modules['avgpool'] = torch.nn.AdaptiveAvgPool2d(output_size = (1,1))
# net._modules['classifier'][0] = torch.nn.Linear(in_features=512, out_features=512, bias=True)
# net._modules['classifier'][3] = torch.nn.Linear(in_features=512, out_features=512, bias=True)
# net._modules['classifier'][6] = torch.nn.Linear(in_features=512, out_features=10, bias=True)
learning_rate = 0.005


cfg = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}

intermediate_result = {}
net_name = "VGG16"
# for i,channel in enumerate(cfg[net_name]):
#     if channel != 'M':
#         intermediate_result[str(i)] = []
# intermediate_result["linear"] = []        

class VGG(nn.Module):
    def __init__(self, vgg_name):
        super(VGG, self).__init__()
        self.features = self._make_layers(cfg[vgg_name])
        self.classifier = nn.Linear(512, 10)
        global intermediate_result

    def forward(self, x):
        seq = self.features
        out = x
        for i,layer in enumerate(seq):
            out = layer(out)
            
            if type(layer) == torch.nn.modules.conv.Conv2d:
                intermediate_result[str(i)] = out
#         out = self.features(x)
        out = out.view(out.size(0), -1)
        intermediate_result["linear"] = out
        out = self.classifier(out)
        return out

    def _make_layers(self, cfg):
        layers = []
        in_channels = 3
        for x in cfg:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                           nn.BatchNorm2d(x),
                           nn.ReLU(inplace=True)]
                in_channels = x
        layers += [nn.AvgPool2d(kernel_size=1, stride=1)]
        return nn.Sequential(*layers)



        
    

net = VGG(net_name)
print(net)



# 定义损失函数和优化器
criterion = torch.nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4)

# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

# 如果有gpu就使用gpu，否则使用cpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

net = net.to(device)
print("load_model")
net = VGG('VGG16').to(device)

optimizer_load = torch.optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4)

checkpoint = torch.load('models/VGG_enhance/p_pull_away_Robust_1_checkpoint.pth')
net.load_state_dict(checkpoint['model_state_dict'])
optimizer_load.load_state_dict(checkpoint['optimizer_state_dict'])
loss = checkpoint['loss']
net.eval()

In [ ]:
    

net.eval()
test_loss = 0
test_acc = 0
test_robust_loss = 0
test_robust_acc = 0
test_n = 0
for i, batch in enumerate(tqdm(test_batches)):
    X, y = batch['input'], batch['target']

    # Random initialization
    if AE_attack == 'none':
        delta = torch.zeros_like(X)
    else:
        delta = attack_pgd(net, X, y, epsilon, pgd_alpha, attack_iters_test, restarts, norm, early_stop=eval_)
    delta = delta.detach()

    robust_output = net(normalize(torch.clamp(X + delta[:X.size(0)], min=lower_limit, max=upper_limit)))
    robust_loss = criterion(robust_output, y)

    output = net(normalize(X))
    loss = criterion(output, y)

    test_robust_loss += robust_loss.item() * y.size(0)
    test_robust_acc += (robust_output.max(1)[1] == y).sum().item()
    test_loss += loss.item() * y.size(0)
    test_acc += (output.max(1)[1] == y).sum().item()
    test_n += y.size(0)

logger.info(f'{"="*20} Test {"="*20}')
logger.info('Loss \t Acc \t Robust Loss \t Robust Acc')
logger.info('%.4f \t %.4f \t %.4f \t %.4f', test_loss/test_n, test_acc/test_n, test_robust_loss/test_n, test_robust_acc/test_n)

In [41]:
from utils_notion import *
from datetime import datetime
ID = "".join(list(filter(str.isdigit, str(datetime.now()))))
model_info_dict = {"ID" : ID,
                   "Dataset" : "CIFAR-10",
                   "Architecture" : "VGG-16",
                   "Backdoor" : "BadNet",
                   "AT" : "AWP"}
createPage(databaseId, headers, model_info_dict)

200
{"object":"page","id":"4f80db26-e56f-4813-8e86-78a03c0ddfe7","created_time":"2022-12-20T21:21:00.000Z","last_edited_time":"2022-12-20T21:21:00.000Z","created_by":{"object":"user","id":"16d62e4d-6d00-4023-a7f7-81897d05a207"},"last_edited_by":{"object":"user","id":"16d62e4d-6d00-4023-a7f7-81897d05a207"},"cover":null,"icon":null,"parent":{"type":"database_id","database_id":"d69f5d0d-93ab-4dc4-b997-5582c0d89bbe"},"archived":false,"properties":{"Architecture":{"id":"%5Cdjo","type":"select","select":{"id":"6d8f5cf5-acec-446a-a9dc-bcc38f19e422","name":"VGG-16","color":"green"}},"AT":{"id":"%60tIR","type":"select","select":{"id":"wnZA","name":"AWP","color":"green"}},"Dataset":{"id":"bRI%40","type":"select","select":{"id":"5e2ed25e-4efe-41d1-8a39-994b4b77e1e4","name":"CIFAR-10","color":"green"}},"Backdoor":{"id":"uVm%60","type":"select","select":{"id":"438ca02b-53e1-4145-baa2-6d4ecc1e0b0b","name":"BadNet","color":"green"}},"ID":{"id":"title","type":"title","title":[{"type":"text","text":{"c

In [28]:
from datetime import datetime
# str(datetime.now()).replace(' ', '')
"".join(list(filter(str.isdigit, str(datetime.now()))))

'20221220160533603446'